<a href="https://colab.research.google.com/github/texaspandaa/NZ/blob/main/Copy_of_%E2%93%82%EF%B8%8FWorking_with_pyLDAvis_Topic_Modelling_Exploration_Tool_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know


https://app.neptune.ai/khuyentran1401/sandbox/n/Topic-modeling-b25db361-8995-42ee-bd50-6f03fa8d5847/65d94d91-43e9-4c21-bee6-8a9bf7584087


https://cloud.datapane.com/reports/Y3Ya6e3/processed-tweets/

# (1) Preparation

## (1.1) Install package

In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


## (2) import library

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd

# (1) pyLDAvis - dp-export dataset

In [ ]:
import pandas as pd
dset_url='https://archive.org/download/misc-dataset/dp-export-tokenized.csv'
tweets = pd.read_csv(dset_url)
tweets = tweets.Tweets.values.tolist()
tweets = [t.split(',') for t in tweets]
print(tweets[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['speak', 'health', 'care', 'reform', 'morning', 'live', 'tweeting', 'allow', 'happen', 'true', 'cover', 'hard', 'gawk', 'son', 'build', 'key', 'question', 'hit', 'plan', 'year', 'hit', 'plan', 'year', 'must', 'read', 'ask', 'orderly', 'generation', 'family', 'can', 'speak', 'branch', 'order', 'specific', 'action', 'declaration', 'administrative', 'orderly', 'transition', 'massive', 'disruption', 'go', 'protester', 'chant', 'really', 'motorcade', 'head', 'protester', 'many', 'slice', 'read', 'work', 'solution', 'trade', 'off', 'view', 'chant', 'singe', 'scene', 'start', 'spread', 'quick', 'thought', 'ask', 'how', 'view', 'grind', 'poster', 'reference', 'interview', 'continue', 'press', 'drug', 'rebate', 'classification', 'will', 'fallout', 'happiness', 'speak', 'see', 'spark', 'teach', 'dem', 'board', 'aca', 'replacement', 'republican', 'should', 'take', 'table', 'ugh', 'scandal', 'behavior', 'incentive', 'donation', "'d", 'sing', 'sit', 'dock', 'rap', 'chicken', 'dance', 'thank', 'day

In [ ]:
# create dictionary
id2word = Dictionary(tweets)

# create corpus
corpus = [id2word.doc2bow(text) for text in tweets]

# test
print(corpus[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 1), (38, 9), (39, 1), (40, 1), (41, 1), (42, 2), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 5), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 5), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 5), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 4), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [ ]:
# test by checking the id and its corresponding word value and frequency
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[("'d", 1),
  ('-', 1),
  ('absolutely', 1),
  ('aca', 3),
  ('act', 1),
  ('action', 2),
  ('add', 2),
  ('administrative', 1),
  ('affordable', 1),
  ('allow', 1),
  ('amazing', 1),
  ('arrive', 2),
  ('ask', 2),
  ('audits', 1),
  ('av', 1),
  ('avoid', 1),
  ('away', 2),
  ('back', 1),
  ('ball', 1),
  ('baseball', 1),
  ('beget', 2),
  ('begin', 1),
  ('behavior', 1),
  ('believe', 1),
  ('bid', 1),
  ('big', 2),
  ('billy', 1),
  ('board', 1),
  ('bout', 1),
  ('branch', 1),
  ('break', 1),
  ('bring', 1),
  ('brother', 1),
  ('build', 1),
  ('call', 2),
  ('can', 1),
  ('cap', 2),
  ('car', 1),
  ('care', 9),
  ('cell', 1),
  ('certainly', 1),
  ('chair', 1),
  ('change', 2),
  ('chant', 2),
  ('chicken', 1),
  ('child', 1),
  ('chip', 1),
  ('choice', 1),
  ('choke', 1),
  ('chuck', 1),
  ('classification', 1),
  ('close', 1),
  ('come', 2),
  ('community', 2),
  ('compare', 2),
  ('competition', 1),
  ('competitively', 1),
  ('compliant', 1),
  ('conclusion', 1),
  ('conferen

In [ ]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Print the Keyword
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.017*"go" + 0.013*"think" + 0.013*"know" + 0.010*"time" + 0.010*"people" + '
  '0.008*"good" + 0.008*"thing" + 0.007*"feel" + 0.007*"need" + 0.007*"get"'),
 (1,
  '0.020*"game" + 0.019*"play" + 0.019*"good" + 0.013*"win" + 0.012*"go" + '
  '0.010*"look" + 0.010*"great" + 0.010*"team" + 0.010*"time" + 0.009*"year"'),
 (2,
  '0.029*"video" + 0.026*"new" + 0.021*"like" + 0.020*"day" + 0.019*"today" + '
  '0.015*"check" + 0.014*"photo" + 0.009*"post" + 0.009*"morning" + '
  '0.009*"year"'),
 (3,
  '0.186*"more" + 0.058*"today" + 0.021*"pisce" + 0.016*"capricorn" + '
  '0.015*"cancer" + 0.015*"aquarius" + 0.013*"arie" + 0.008*"feel" + '
  '0.008*"gemini" + 0.006*"idea"'),
 (4,
  '0.017*"great" + 0.011*"new" + 0.010*"thank" + 0.010*"work" + 0.008*"good" + '
  '0.008*"look" + 0.007*"how" + 0.006*"learn" + 0.005*"need" + 0.005*"year"'),
 (5,
  '0.028*"thank" + 0.026*"love" + 0.017*"good" + 0.013*"day" + 0.010*"year" + '
  '0.010*"look" + 0.010*"happy" + 0.010*"great" + 0.010*"time" + 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.3536447840003421


In [ ]:

#Creating Topic Distance Visualization
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.155750 -0.051116       1        1  19.906386
5     -0.135708 -0.121613       2        1  19.478601
8     -0.115950  0.132326       3        1  18.752256
4     -0.070989  0.039456       4        1  14.409833
1     -0.144890 -0.087680       5        1   8.715035
6     -0.043444  0.249516       6        1   8.327551
2     -0.051863 -0.173048       7        1   6.981709
3      0.201999 -0.037246       8        1   1.773532
7      0.130644  0.112431       9        1   1.102508
9      0.385951 -0.063025      10        1   0.552590, topic_info=        Term          Freq         Total Category  logprob  loglift
339    thank  22182.000000  22182.000000  Default  30.0000  30.0000
231   people  16332.000000  16332.000000  Default  29.0000  29.0000
200     more  10057.000000  10057.000000  Default  28.0000  28.0000
627    today  16010.000000  16010.000000  Default  27.0000  27.0000
119   follow   5272.000000   5272.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1769  medium     45.826305   2974.183679  Topic10  -5.8274   1.0254
155      how     47.672150   7825.797146  Topic10  -5.7879   0.0975
649      why     40.584354   5218.615192  Topic10  -5.9488   0.3417
868     free     38.835995   4025.225937  Topic10  -5.9929   0.5573
1814  social     36.630479   2251.922482  Topic10  -6.0513   1.0797

[808 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1         1  0.055080     -
1         2  0.052861     -
1         3  0.411788     -
1         4  0.212451     -
1         5  0.007869     -
...     ...       ...   ...
378       9  0.008161  year
9344      5  0.993980   zip
3684      1  0.028362     °
3684      7  0.212714     °
3684      9  0.758679     °

[2692 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 9, 5, 2, 7, 3, 4, 8, 10])

# (2) pyLDAvis - airline dataset

In [ ]:
dset_url='https://archive.org/download/misc-dataset/airline-tweets-ngram.csv'
airline_df = pd.read_csv(dset_url) #Change this with the name of your downloaded file

airline_df['grams'] = airline_df['gram1_word_gt_1'].astype('string') + ','+airline_df['gram2_word_gt_1'].astype('string')+','+airline_df['gram3_word_gt_1'].astype('string')
# replace one or more commas with a single space character
airline_df['grams'] = airline_df['grams'].astype('string').replace(r'[,\s]+', r' ', regex=True)
airline_df.dropna(inplace=True)

airline_grams =[s.split() for s in airline_df.grams.values.tolist()]

print(airline_grams[0:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['seriously', 'would', 'pay', 'flight', 'seat', 'play', 'really', 'bad', 'thing', 'fly', 'va', 'would_pay', 'pay_flight', 'flight_seat', 'seat_not', 'not_play', 'really_bad', 'bad_thing', 'thing_fly', 'fly_va', 'bad_thing_fly']]


In [ ]:
# create dictionary
airline_id2word = Dictionary(airline_grams)

# create corpus
airline_corpus = [airline_id2word.doc2bow(text) for text in airline_grams]

# test
print(airline_corpus[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [ ]:
# test by checking the id and its corresponding word value and frequency
[[(airline_id2word[i], freq) for i, freq in doc] for doc in airline_corpus[0:1]]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('bad', 1),
  ('bad_thing', 1),
  ('bad_thing_fly', 1),
  ('flight', 1),
  ('flight_seat', 1),
  ('fly', 1),
  ('fly_va', 1),
  ('not_play', 1),
  ('pay', 1),
  ('pay_flight', 1),
  ('play', 1),
  ('really', 1),
  ('really_bad', 1),
  ('seat', 1),
  ('seat_not', 1),
  ('seriously', 1),
  ('thing', 1),
  ('thing_fly', 1),
  ('va', 1),
  ('would', 1),
  ('would_pay', 1)]]

In [ ]:
# Build LDA model
airline_lda_model = LdaModel(corpus=airline_corpus,
                   id2word=airline_id2word,
                   num_topics=10,
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Print the Keyword
pprint(airline_lda_model.print_topics())
doc_lda = airline_lda_model[airline_corpus]

[(0,
  '0.058*"call" + 0.034*"cannot" + 0.033*"get" + 0.025*"help" + 0.023*"flight" '
  '+ 0.023*"phone" + 0.021*"still" + 0.018*"make" + 0.018*"wait" + '
  '0.015*"call_back"'),
 (1,
  '0.166*"flight" + 0.077*"cancel" + 0.043*"flightled" + '
  '0.042*"cancel_flightled" + 0.025*"tomorrow" + 0.022*"cancel_flight" + '
  '0.019*"dfw" + 0.015*"need" + 0.014*"miss" + 0.014*"home"'),
 (2,
  '0.053*"get" + 0.044*"back" + 0.031*"flight_cancel" + 0.028*"work" + '
  '0.024*"us" + 0.024*"flight_cancel_flightled" + 0.022*"thank" + 0.022*"you" '
  '+ 0.019*"know" + 0.018*"let"'),
 (3,
  '0.070*"hold" + 0.037*"weather" + 0.030*"please" + 0.023*"email" + '
  '0.022*"hour" + 0.021*"response" + 0.015*"rebook" + 0.015*"info" + '
  '0.014*"do" + 0.013*"dm"'),
 (4,
  '0.035*"hour" + 0.028*"delay" + 0.027*"one" + 0.023*"time" + 0.021*"bad" + '
  '0.021*"even" + 0.016*"ever" + 0.015*"line" + 0.014*"would" + 0.014*"me"'),
 (5,
  '0.037*"gate" + 0.033*"change" + 0.027*"late" + 0.026*"agent" + '
  '0.023*"leav

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Compute Coherence Score
airline_coherence_model_lda = CoherenceModel(model=airline_lda_model, texts=airline_grams, dictionary=airline_id2word, coherence='c_v')
airline_coherence_lda = airline_coherence_model_lda.get_coherence()
print('\nCoherence Score: ', airline_coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.3016957987636324


In [ ]:
#Creating Topic Distance Visualization
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(airline_lda_model, airline_corpus, airline_id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.183055  0.202947       1        1  17.007607
1      0.270859 -0.252698       2        1  14.419754
4     -0.172757 -0.128032       3        1  12.489746
6     -0.097420 -0.063627       4        1  10.663338
2     -0.003140  0.167719       5        1  10.005126
8     -0.085044 -0.046912       6        1   8.417625
5      0.053564  0.032261       7        1   7.529030
7     -0.025443  0.086639       8        1   6.694856
3     -0.100057 -0.000415       9        1   6.538820
9     -0.023618  0.002119      10        1   6.234097, topic_info=             Term         Freq        Total Category  logprob  loglift
3          flight  3425.000000  3425.000000  Default  30.0000  30.0000
375        cancel  1307.000000  1307.000000  Default  29.0000  29.0000
263          call  1160.000000  1160.000000  Default  28.0000  28.0000
215           get  1797.000000  1797.000000  Default  27.0000  27.0000
292      customer   744.000000   744.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2889   ridiculous    45.955859    48.023921  Topic10  -5.0655   2.7311
283          need   181.864550   458.115131  Topic10  -3.6899   1.8513
106          help   250.534363   881.292179  Topic10  -3.3695   1.5173
288   reservation    56.636526   287.327346  Topic10  -4.8565   1.1512
399          hour    61.859833  1046.114266  Topic10  -4.7683  -0.0528

[433 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
173       7  0.976203          a
3626      4  0.997317         aa
1001      8  0.986197       able
3980      8  0.966653   able_get
1722      9  0.988233   actually
...     ...       ...        ...
1142      7  0.966382        wtf
33        8  0.993627        yes
640       1  0.987235  yesterday
75        7  0.994477        yet
133       5  0.995732        you

[508 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 7, 3, 9, 6, 8, 4, 10])